In [1]:
# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
get_ipython().magic('env CUDA_VISIBLE_DEVICES = ')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import os.path
import random
import time
from collections import OrderedDict
import io
from datetime import datetime
import gc # garbage collector
import math
import sys
from collections import defaultdict
import re
import logging
from tensorboard_logger import configure, log_value

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
get_ipython().magic('matplotlib inline')
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')

env: CUDA_VISIBLE_DEVICES=


## Write a pandas dataframe to disk as gunzip compressed csv
- df.to_csv('dfsavename.csv.gz', compression='gzip')

## Read from disk
- df = pd.read_csv('dfsavename.csv.gz', compression='gzip')

## Magic useful
- %%timeit for the whole cell
- %timeit for the specific line
- %%latex to render the cell as a block of latex
- %prun and %%prun

In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/WSDM/'
HDF_FILENAME = DATASET_PATH + 'datas.h5'
HDF_FILENAME_TEMPSAVE = DATASET_PATH + 'datas_temp.h5'
SUBMISSION_FILENAME = DATASET_PATH + 'submission_{}.csv'
VALIDATION_INDICE = DATASET_PATH + 'validation_indice.csv'

LOG_DIR = DATASET_PATH + 'music_logs'

In [3]:
def set_logging(logger_name, logger_file_name):
    log = logging.getLogger(logger_name)
    log.setLevel(logging.DEBUG)

    # create formatter and add it to the handlers
    print_formatter = logging.Formatter('%(message)s')
    file_formatter = logging.Formatter('%(asctime)s - %(name)s_%(levelname)s: %(message)s')

    # create file handler which logs even debug messages
    fh = logging.FileHandler(logger_file_name, mode='w')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(file_formatter)
    log.addHandler(fh)
    # both output to console and file
    consoleHandler = logging.StreamHandler()
    consoleHandler.setFormatter(print_formatter)
    log.addHandler(consoleHandler)
    
    return log

In [4]:
log = set_logging('MUSIC', DATASET_PATH + 'music_pytorch.log')
log.info('here is an info message.')

here is an info message.


In [5]:
# store_test = pd.HDFStore(HDF_FILENAME)
# test_id =  store_test['test_id']
# store_test.close()

# gc.collect()
# train_use = pd.read_csv(DATASET_PATH + 'msno_artist_train.csv', compression='gzip')
# gc.collect()
# validation_use = pd.read_csv(DATASET_PATH + 'msno_artist_val.csv', compression='gzip')
# test = pd.read_csv(DATASET_PATH + 'msno_artist_test.csv', compression='gzip')
# gc.collect()

# temp_data = pd.concat([train_use, validation_use, test], axis=0, join="outer")
# hist = temp_data.groupby(['msno', 'artist_name'])['msno'].agg(['count']).reset_index(drop=False)

In [6]:
# all_neg = None
# start_time = time.time()
# for msno_index in range(34403):
#     keep_list = list(temp_data[temp_data['msno']==msno_index]['artist_name'].values)
#     neg_sample_artist = list()
#     for _ in range(5):
#         rand_int = np.random.random_integers(0, 45045)
#         while rand_int in keep_list:
#             rand_int = np.random.random_integers(0, 45045)
#         keep_list.append(rand_int)
#         neg_sample_artist.append(rand_int)
#     temp_data = pd.DataFrame(data={'msno': [msno_index]*5, 'artist_name': neg_sample_artist}).reindex(columns=['msno', 'artist_name'])

#     if all_neg is None:
#         all_neg = temp_data
#     else:
#         all_neg = pd.concat([all_neg, temp_data], join="outer", axis = 0)
#     if msno_index % 1000 == 0:
#         time_elapsed = time.time() - start_time
#         start_time = time.time()
#         print('cur index: {}'.format(msno_index))
#         print('Current Speed: {:5.3f}sec/1000'.format(time_elapsed))   
# print('Finished.')        
# all_neg = all_neg.reset_index(drop=True)    
# all_neg['count'] = 0
# #print(all_neg)
# hist = pd.concat([hist, all_neg], join="outer", axis = 0).sample(frac=1)
# hist = hist.reset_index(drop=True)
# #print(hist)
# hist.to_csv(DATASET_PATH + 'all_samples.csv', index = False, compression='gzip')
# print('Saved.')        

- msno: 0-34402
- artist_name: 0-45045
- count:0, 1, 2-8, 9-496

In [7]:
# # temp_data[temp_data['msno']==0]['artist_name'].values
# # print(len(temp_data))

# print(len(hist[hist['count']==0]))
# print(len(hist[hist['count']==1]))
# print(len(hist[hist['count'].map(lambda x: x>=2 and x < 8)]))
# print(len(hist[hist['count'].map(lambda x: x>=9)]))

In [8]:
# org_hist = hist.copy(deep=True)
# count_0 = org_hist[org_hist['count']==0]
# count_1 = org_hist[org_hist['count']==1]
# count_29 = org_hist.loc[(org_hist["count"] > 1) & (org_hist["count"] < 9), ['msno', 'artist_name', 'count']].copy(deep=True)
# count_9 = org_hist.loc[(org_hist["count"] > 8), ['msno', 'artist_name', 'count']].copy(deep=True)

# for _ in range(12):
#     hist = pd.concat([hist, count_0], join="outer", axis = 0)

# for _ in range(1):
#     hist = pd.concat([hist, count_29], join="outer", axis = 0)

# for _ in range(12):
#     hist = pd.concat([hist, count_9], join="outer", axis = 0)

# print('Finished.')  
# hist = hist.sample(frac=1)
# hist = hist.reset_index(drop=True)
# hist.to_csv(DATASET_PATH + 'all_samples_balance.csv', index = False, compression='gzip')
# print('Saved.')    
# print(len(hist))

In [9]:
all_data = pd.read_csv(DATASET_PATH + 'all_samples_balance.csv', compression='gzip')
#train_data = all_data[:128]
train_data = all_data[:-600000]
val_data = all_data[-600000:]
#val_data = all_data[-64:]
train_examples_num = len(train_data)
val_examples_num = len(val_data)
print(train_examples_num, val_examples_num)
gc.collect()

9065840 600000


0

In [10]:
from tensorboard_logger import configure, log_value
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.autograd import Variable
import torch

num_epoches = 20
embedding_dim = 64
batch_size = 64

msno_count = 34403
artist_count = 45046

time_interval_to_save = 3000
log_step_interval = 1000

num_steps_per_train_epoch = int(train_examples_num / (batch_size)) + 1
num_steps_per_val_epoch = int(val_examples_num / (batch_size)) + 1

class MusicDataset(Dataset):
    def __init__(self, data_input):
        super(MusicDataset, self).__init__()
        self._msno = data_input['msno'].values
        self._artist = data_input['artist_name'].values
        self._target = data_input['count'].apply(lambda x : MusicDataset.encode_count(x)).values
        self._num_examples = len(data_input)

    @staticmethod
    def encode_count(count):
        if count < 2:
            return count
        elif count > 8:
            return 3
        else:
            return 2  
        
    def __len__(self):
        return self._num_examples

    def __getitem__(self, idx):
        return (self._msno[idx], self._artist[idx], self._target[idx])
    

class NCF(nn.Module):
    def __init__(self):
        super(NCF, self).__init__()
        self._msno_embedding = nn.Embedding(msno_count, embedding_dim)
        self._artist_embedding = nn.Embedding(artist_count, embedding_dim)
    
        self.layer1 = nn.Sequential(nn.Linear(2 * embedding_dim, 64), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(64, 32), nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(32, 16), nn.ReLU(True))
        self.dropout = nn.Dropout(0.5)
        #self.layer4 = nn.Sequential(nn.Linear(16, 4), nn.ReLU(True))
        self.layer4 = nn.Linear(16, 4)
        #self.softmax = nn.LogSoftmax()
        #self.softmax = nn.Softmax()

    def forward(self, msno_id, artist_id):
        msno_embedded = self._msno_embedding(msno_id)
        artist_embedded = self._artist_embedding(artist_id)
        input_x = torch.cat((msno_embedded, artist_embedded), 1)
        input_x = self.layer1(input_x)
        input_x = self.layer2(input_x)
        input_x = self.layer3(input_x)
        input_x = self.dropout(input_x)
        input_x = self.layer4(input_x)
        
        return input_x#self.softmax(input_x)


def save_checkpoint(log_dir, model_state_dict, other_state_dict):
    checkpoint_file_name = os.path.join(log_dir, 'checkpoint')
    os.makedirs(log_dir, exist_ok=True)
    
    checkpoint_list = list()
    if os.path.exists(checkpoint_file_name):
        with open(checkpoint_file_name, 'r') as checkpoint_file:
            for index, line in enumerate(checkpoint_file):
                if index == 0: continue
                if line.strip() != '':
                    checkpoint_list.append(line.strip())
        
    model_save_name = (checkpoint_file_name + '_pytorch_state_{}.pth').format(datetime.now().strftime('%Y-%m-%d_%H_%M_%S'))
    checkpoint_list.append(model_save_name)
    
    torch.save(model_state_dict, model_save_name)
    torch.save(other_state_dict, model_save_name.replace('state','others'))
    
    log.info('model saved: {}.'.format(model_save_name))
    
    with open(checkpoint_file_name, 'w') as outfile:
        outfile.write(model_save_name+'\n')
        for line in checkpoint_list:
            outfile.write(line+'\n')

def restore_from_checkpoint(log_dir, model):
    checkpoint_filename = None
    if os.path.exists(os.path.join(log_dir, 'checkpoint')):
        with open(os.path.join(log_dir, 'checkpoint'), 'r') as checkpoint_file:
            for _, line in enumerate(checkpoint_file):
                if line.strip() != '':
                    checkpoint_filename = line.strip()
                    break
    if (not os.path.isdir(log_dir)) or (checkpoint_filename is None):
        return None
    
    model.load_state_dict(torch.load(checkpoint_filename, map_location=lambda storage, loc: storage))    

    log.info('model resotred from: {}.'.format(checkpoint_filename))
    
    return torch.load(checkpoint_filename.replace('state','others'))    
 
def exp_lr_scheduler(optimizer, epoch, init_lr=0.01, lr_decay_epoch=1):
    lr = init_lr * (0.27**(epoch // lr_decay_epoch))

    #if lr < 0.002: lr = 0.002
    if lr < 0.000002: lr = 0.000002
        
    if epoch % lr_decay_epoch == 0:
        log.info('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

NCF_encoder = NCF()

if torch.cuda.is_available():
    NCF_encoder = NCF_encoder.cuda()

#criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(NCF_encoder.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0002)


others_state = restore_from_checkpoint(LOG_DIR, NCF_encoder)
start_iter = 1
start_epoch = 0
if others_state is not None:
    start_iter  = others_state['iter']
    start_epoch = others_state['epoch']
    optimizer.load_state_dict(others_state['optimizer'])
    
# initialize tensorboard_logger
configure(LOG_DIR + "/tensorboard_logger_{}".format(datetime.now().strftime('%Y-%m-%d_%H_%M_%S')), flush_secs=120)

train_set = MusicDataset(train_data)
val_set = MusicDataset(val_data)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers = 4, drop_last=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers = 4, drop_last=False)
    
#target_onehot = torch.FloatTensor(batch_size, 4)
save_start_time = time.time()
for epoch in range(num_epoches):
    if epoch < start_epoch: continue
    log.info('epoch {} of {} start...'.format(epoch, num_epoches))
    log.info('start from {} of epoch {}.'.format(start_iter, epoch))
    NCF_encoder.train()
    smooth_train_num_correct = 0
    last_train_step = start_iter
    step_start_time = time.time()
    
    optimizer = exp_lr_scheduler(optimizer, epoch)
    
    for index, data in enumerate(train_loader, start_iter):
        msno, artist, target = data

        msno_tensor = torch.LongTensor(msno)
        artist_tensor = torch.LongTensor(artist)
        target_tensor = torch.LongTensor(target)
            
        if torch.cuda.is_available():
            msno_in = Variable(msno_tensor).cuda()
            artist_in = Variable(artist_tensor).cuda()
            target_in = Variable(target_tensor).cuda()
        else:
            msno_in = Variable(msno_tensor)
            artist_in = Variable(artist_tensor)
            target_in = Variable(target_tensor)
           
        target_logits = NCF_encoder(msno_in, artist_in)
        target_softmax = nn.Softmax()(target_logits)
        loss = criterion(target_logits, target_in)
        _, batch_top1 = torch.topk(target_softmax, 1)
        batch_top1 = batch_top1.type(torch.LongTensor)
        if torch.cuda.is_available():
            batch_top1 = batch_top1.cuda()
        num_correct = (target_in == batch_top1.view(1, -1)).sum()
#         target_log_softmax = NCF_encoder(msno_in, artist_in)
#         loss = criterion(target_log_softmax, target_in)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #print(target_softmax.data.numpy())
        #print(batch_top1.data.numpy())
        #print(target_in.data.numpy())
#         print((target_in == (batch_top1.view(1, -1))).data.numpy())
#         print(num_correct.data[0])
        
        batch_acc  = num_correct.data[0]/batch_size
        batch_loss = loss.data[0]
        smooth_train_num_correct += num_correct.data[0]

        smooth_train_acc = smooth_train_num_correct/((index-start_iter+1)*batch_size)    
        if index % log_step_interval == 0:
        #if True:
            # to tensorboard
            cur_step = int(index + epoch * num_steps_per_train_epoch)
            log_value('train/batch_loss', batch_loss, cur_step)
            log_value('train/sec_per_step', (time.time()-step_start_time)/(index-last_train_step+1), cur_step)
            log_value('train/learning_rate', optimizer.param_groups[0]['lr'], cur_step)
            log_value('train/batch_acc', batch_acc, cur_step)
            log_value('train/smooth_train_acc', smooth_train_acc, cur_step)
            # to stdout
            log.info('####### train logging #######')
            log.info('\ttrain/current_step: {} of {}, cur_epoch: {}'.format(cur_step, num_steps_per_train_epoch*(epoch+1), epoch))
            log.info('\ttrain/current_lr: {}'.format(optimizer.param_groups[0]['lr']))
            log.info('\ttrain/sec_per_step: {:.3f}'.format((time.time()-step_start_time)/(index-last_train_step+1)))
            log.info('\ttrain/batch_loss: {:.3f}'.format(batch_loss))
            log.info('\ttrain/batch_acc: {:.3f}'.format(batch_acc*100))
            log.info('\ttrain/smooth_train_acc: {:.3f}'.format(smooth_train_acc*100))
            last_train_step = index
            step_start_time = time.time()
        
        frac_time_passed = (time.time() - save_start_time + 1) / time_interval_to_save
        if (frac_time_passed - int(frac_time_passed)) < 1e-6:
        #if int(time.time() - save_start_time + 1) % time_interval_to_save == 0:  
            save_checkpoint(LOG_DIR, NCF_encoder.state_dict(), {'optimizer': optimizer.state_dict(),
                                                    'iter': index, 'epoch': epoch })
            
    log.info('epoch {} finished.'.format(epoch)) 
    # save model after each epoch
    save_checkpoint(LOG_DIR, NCF_encoder.state_dict(), {'optimizer': optimizer.state_dict(),
                                                    'iter': 0, 'epoch': epoch })
    # reset start_iter
    start_iter = 1
    
    # check on validation every epoches            
    NCF_encoder.eval()
    last_val_step = 1
    smooth_validation_num_correct = 0
    step_start_time = time.time()
    #if False:
    for index, data in enumerate(val_loader, 1):
        msno, artist, target = data
        
        msno_tensor = torch.LongTensor(msno)
        artist_tensor = torch.LongTensor(artist)
        target_tensor = torch.LongTensor(target)
            
        if torch.cuda.is_available():
            msno_in = Variable(msno_tensor).cuda()
            artist_in = Variable(artist_tensor).cuda()
            target_in = Variable(target_tensor).cuda()
        else:
            msno_in = Variable(msno_tensor)
            artist_in = Variable(artist_tensor)
            target_in = Variable(target_tensor)
           
        _, batch_top1 = torch.topk(nn.Softmax()(NCF_encoder(msno_in, artist_in)), 1)
        batch_top1 = batch_top1.type(torch.LongTensor)
        if torch.cuda.is_available():
            batch_top1 = batch_top1.cuda()
        num_correct = (target_in == batch_top1.view(1, -1)).sum()
        
        batch_acc  = num_correct.data[0]/batch_size
        smooth_validation_num_correct += num_correct.data[0]

        smooth_validation_acc = smooth_validation_num_correct/((index+1)*batch_size) 

        if index % log_step_interval == 0:
            cur_step = int(index + epoch * num_steps_per_val_epoch)
            log_value('validation/batch_acc', batch_acc, cur_step)
            log_value('validation/smooth_validation_acc', smooth_validation_acc, cur_step)
            log.info('####### validation logging #######')
            log.info('\tvalidation/current_step: {}/{}'.format(index, num_steps_per_val_epoch))
            log.info('\tvalidation/sec_per_step: {:.3f}'.format((time.time()-step_start_time)/(index-last_val_step+1)))
            log.info('\tvalidation/batch_acc: {:.3f}'.format(batch_acc*100))
            log.info('\tvalidation/smooth_validation_acc: {:.3f}'.format(smooth_validation_acc*100))
            last_val_step = index
            step_start_time = time.time()
log.info('Done: {}'.format(datetime.now().strftime('%Y-%m-%d_%H_%M_%S')))

epoch 0 of 20 start...
start from 1 of epoch 0.
LR is set to 0.01
####### train logging #######
	train/current_step: 1000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.041
	train/batch_loss: 1.384
	train/batch_acc: 29.688
	train/smooth_train_acc: 29.083
####### train logging #######
	train/current_step: 2000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.042
	train/batch_loss: 1.283
	train/batch_acc: 37.500
	train/smooth_train_acc: 32.334
####### train logging #######
	train/current_step: 3000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.043
	train/batch_loss: 1.325
	train/batch_acc: 29.688
	train/smooth_train_acc: 34.542
####### train logging #######
	train/current_step: 4000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.043
	train/batch_loss: 1.278
	train/batch_acc: 34.375
	train/smooth_train_acc: 36.127
####### train logging #######
	train/current_step: 5000 of 141654, cur_epoch: 0
	t

	train/current_step: 39000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.044
	train/batch_loss: 1.122
	train/batch_acc: 51.562
	train/smooth_train_acc: 48.460
####### train logging #######
	train/current_step: 40000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.044
	train/batch_loss: 1.094
	train/batch_acc: 54.688
	train/smooth_train_acc: 48.587
####### train logging #######
	train/current_step: 41000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.044
	train/batch_loss: 1.025
	train/batch_acc: 53.125
	train/smooth_train_acc: 48.719
####### train logging #######
	train/current_step: 42000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.044
	train/batch_loss: 1.006
	train/batch_acc: 53.125
	train/smooth_train_acc: 48.839
####### train logging #######
	train/current_step: 43000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.045
	train/batch_loss: 0.902
	train/batch_acc:

	train/sec_per_step: 0.044
	train/batch_loss: 0.909
	train/batch_acc: 60.938
	train/smooth_train_acc: 51.951
####### train logging #######
	train/current_step: 78000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.034
	train/batch_loss: 0.891
	train/batch_acc: 64.062
	train/smooth_train_acc: 52.020
####### train logging #######
	train/current_step: 79000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.036
	train/batch_loss: 1.190
	train/batch_acc: 53.125
	train/smooth_train_acc: 52.086
####### train logging #######
	train/current_step: 80000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.038
	train/batch_loss: 1.002
	train/batch_acc: 56.250
	train/smooth_train_acc: 52.152
model saved: /media/rs/0E06CD1706CD0127/Kapok/WSDM/music_logs/checkpoint_pytorch_state_2017-11-17_12_34_01.pth.
model saved: /media/rs/0E06CD1706CD0127/Kapok/WSDM/music_logs/checkpoint_pytorch_state_2017-11-17_12_34_01.pth.
model saved: /media/rs/0E

	train/batch_loss: 0.923
	train/batch_acc: 46.875
	train/smooth_train_acc: 53.824
####### train logging #######
	train/current_step: 114000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.044
	train/batch_loss: 0.888
	train/batch_acc: 60.938
	train/smooth_train_acc: 53.864
####### train logging #######
	train/current_step: 115000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.055
	train/batch_loss: 0.885
	train/batch_acc: 64.062
	train/smooth_train_acc: 53.903
####### train logging #######
	train/current_step: 116000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.060
	train/batch_loss: 0.892
	train/batch_acc: 70.312
	train/smooth_train_acc: 53.942
####### train logging #######
	train/current_step: 117000 of 141654, cur_epoch: 0
	train/current_lr: 0.01
	train/sec_per_step: 0.064
	train/batch_loss: 0.828
	train/batch_acc: 62.500
	train/smooth_train_acc: 53.982
####### train logging #######
	train/current_step: 118000 

	train/sec_per_step: 0.035
	train/batch_loss: 0.832
	train/batch_acc: 67.188
	train/smooth_train_acc: 61.251
####### train logging #######
	train/current_step: 144654 of 283308, cur_epoch: 1
	train/current_lr: 0.0027
	train/sec_per_step: 0.035
	train/batch_loss: 0.804
	train/batch_acc: 64.062
	train/smooth_train_acc: 61.379
####### train logging #######
	train/current_step: 145654 of 283308, cur_epoch: 1
	train/current_lr: 0.0027
	train/sec_per_step: 0.037
	train/batch_loss: 0.903
	train/batch_acc: 65.625
	train/smooth_train_acc: 61.534
####### train logging #######
	train/current_step: 146654 of 283308, cur_epoch: 1
	train/current_lr: 0.0027
	train/sec_per_step: 0.038
	train/batch_loss: 0.987
	train/batch_acc: 59.375
	train/smooth_train_acc: 61.518
####### train logging #######
	train/current_step: 147654 of 283308, cur_epoch: 1
	train/current_lr: 0.0027
	train/sec_per_step: 0.034
	train/batch_loss: 0.844
	train/batch_acc: 64.062
	train/smooth_train_acc: 61.596
####### train logging #

...

	train/batch_acc: 59.375
	train/smooth_train_acc: 71.968
####### train logging #######
	train/current_step: 1685194 of 1699848, cur_epoch: 11
	train/current_lr: 2e-06
	train/sec_per_step: 0.056
	train/batch_loss: 0.541
	train/batch_acc: 79.688
	train/smooth_train_acc: 71.970
####### train logging #######
	train/current_step: 1686194 of 1699848, cur_epoch: 11
	train/current_lr: 2e-06
	train/sec_per_step: 0.054
	train/batch_loss: 0.735
	train/batch_acc: 73.438
	train/smooth_train_acc: 71.971
####### train logging #######
	train/current_step: 1687194 of 1699848, cur_epoch: 11
	train/current_lr: 2e-06
	train/sec_per_step: 0.050
	train/batch_loss: 0.586
	train/batch_acc: 76.562
	train/smooth_train_acc: 71.972
####### train logging #######
	train/current_step: 1688194 of 1699848, cur_epoch: 11
	train/current_lr: 2e-06
	train/sec_per_step: 0.055
	train/batch_loss: 0.612
	train/batch_acc: 70.312
	train/smooth_train_acc: 71.973
####### train logging #######
	train/current_step: 1689194 of 16998

	train/current_step: 1714848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.057
	train/batch_loss: 0.549
	train/batch_acc: 76.562
	train/smooth_train_acc: 71.956
####### train logging #######
	train/current_step: 1715848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.059
	train/batch_loss: 0.549
	train/batch_acc: 76.562
	train/smooth_train_acc: 71.959
####### train logging #######
	train/current_step: 1716848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.057
	train/batch_loss: 0.621
	train/batch_acc: 75.000
	train/smooth_train_acc: 71.969
####### train logging #######
	train/current_step: 1717848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.059
	train/batch_loss: 0.553
	train/batch_acc: 71.875
	train/smooth_train_acc: 71.958
####### train logging #######
	train/current_step: 1718848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.059
	train/batch_loss

	train/current_step: 1750848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.040
	train/batch_loss: 0.572
	train/batch_acc: 81.250
	train/smooth_train_acc: 71.980
####### train logging #######
	train/current_step: 1751848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.042
	train/batch_loss: 0.706
	train/batch_acc: 67.188
	train/smooth_train_acc: 71.979
####### train logging #######
	train/current_step: 1752848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.042
	train/batch_loss: 0.560
	train/batch_acc: 78.125
	train/smooth_train_acc: 71.974
####### train logging #######
	train/current_step: 1753848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.042
	train/batch_loss: 0.633
	train/batch_acc: 67.188
	train/smooth_train_acc: 71.977
####### train logging #######
	train/current_step: 1754848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.041
	train/batch_loss

	train/batch_loss: 0.620
	train/batch_acc: 73.438
	train/smooth_train_acc: 71.977
####### train logging #######
	train/current_step: 1788848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.044
	train/batch_loss: 0.701
	train/batch_acc: 68.750
	train/smooth_train_acc: 71.980
####### train logging #######
	train/current_step: 1789848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.042
	train/batch_loss: 0.633
	train/batch_acc: 71.875
	train/smooth_train_acc: 71.983
####### train logging #######
	train/current_step: 1790848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.041
	train/batch_loss: 0.542
	train/batch_acc: 78.125
	train/smooth_train_acc: 71.981
####### train logging #######
	train/current_step: 1791848 of 1841502, cur_epoch: 12
	train/current_lr: 2e-06
	train/sec_per_step: 0.042
	train/batch_loss: 0.632
	train/batch_acc: 73.438
	train/smooth_train_acc: 71.983
model saved: /media/rs/0E06CD1706CD0127/Ka

KeyboardInterrupt: 